In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

## Lectura y Preparación del Dataset

In [2]:
data = pd.read_csv('dataset_final.csv', index_col=0)

In [3]:
from sklearn.preprocessing import LabelEncoder

def data_preparation(df, window=3, neutral=True):
    
    # armar las windows para el armado de las features 
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    for col in cols1:
        df[col+'w'] = df[col].rolling(window=window).mean()
    
    # shifteo un día el retorno. las noticias deben estimar el retorno del día posterior.
    df['exc_ret']  = df['exc_ret'].shift(periods=-1)
    df['exc_ret%'] = df['exc_ret%'].shift(periods=-1) # por si lo necesito 
    
    # elimino las filas que quedan con nan por efecto del rolling window
    df.dropna(axis=0, inplace=True)
    
    # encoding del label que indica si hay retorno positivo (1) o negativo (0)
    le = LabelEncoder()
    df['exc_ret'] = le.fit_transform(df['exc_ret'])
    
    # split en train y test por fecha
    cols2 = [x+'w' for x in cols1]
    
    cut_date = '2018-10-31'
    
    X  = df.loc[df.date <= cut_date, cols2]
    y  = df.loc[df.date <= cut_date, 'exc_ret']
    
    Xt = df.loc[df.date > cut_date, cols2]
    yt = df.loc[df.date > cut_date, 'exc_ret']
    
    return X, Xt, y, yt

## Entrenamiento de Classifiers

In [4]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.svm import SVC

In [5]:
def train_classifier(X, Xt, y, yt):
    
    #model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression',
    #              'Catboost', 'Support Vector']
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  #LogisticRegression(),
                  CatBoostClassifier(),
                  #SVC(probability=True)
                 ]
    
    # Random Forest 
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    # XGBoost y Lightgbm
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    # Logistic Regression
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    # Catboost
    params4 = {}
    
    # SVC
    params5 = {
        "C"           : st.uniform(0.1, 10.),
        #"kernel"      : ['lbf', 'linear']
    }
    
    #model_params = [params1, params2, params2, params3, params4, params5]
    model_params = [params1, params2, params2, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    #kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
    tsp   = TimeSeriesSplit(n_splits=6)
    
    for name, mod, params in zip(model_name, model_init, model_params):
        
        model = mod

        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=tsp,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results

In [6]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = data_preparation(data, window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     #'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    9.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


...

183:	learn: 0.6033512	total: 1.39s	remaining: 6.18s
184:	learn: 0.6028060	total: 1.4s	remaining: 6.17s
185:	learn: 0.6023548	total: 1.41s	remaining: 6.16s
186:	learn: 0.6018065	total: 1.42s	remaining: 6.16s
187:	learn: 0.6013995	total: 1.42s	remaining: 6.15s
188:	learn: 0.6012559	total: 1.43s	remaining: 6.14s
189:	learn: 0.6010778	total: 1.44s	remaining: 6.13s
190:	learn: 0.6008205	total: 1.45s	remaining: 6.13s
191:	learn: 0.6000747	total: 1.45s	remaining: 6.12s
192:	learn: 0.5996861	total: 1.46s	remaining: 6.11s
193:	learn: 0.5993369	total: 1.47s	remaining: 6.11s
194:	learn: 0.5984448	total: 1.48s	remaining: 6.11s
195:	learn: 0.5983326	total: 1.49s	remaining: 6.1s
196:	learn: 0.5981381	total: 1.5s	remaining: 6.09s
197:	learn: 0.5979296	total: 1.5s	remaining: 6.09s
198:	learn: 0.5975146	total: 1.51s	remaining: 6.08s
199:	learn: 0.5973815	total: 1.52s	remaining: 6.07s
200:	learn: 0.5970621	total: 1.53s	remaining: 6.07s
201:	learn: 0.5968204	total: 1.53s	remaining: 6.06s
202:	learn: 0.59

366:	learn: 0.5608921	total: 2.78s	remaining: 4.79s
367:	learn: 0.5607359	total: 2.79s	remaining: 4.78s
368:	learn: 0.5606925	total: 2.79s	remaining: 4.78s
369:	learn: 0.5603799	total: 2.8s	remaining: 4.77s
370:	learn: 0.5598526	total: 2.81s	remaining: 4.76s
371:	learn: 0.5594025	total: 2.81s	remaining: 4.75s
372:	learn: 0.5590493	total: 2.82s	remaining: 4.74s
373:	learn: 0.5587569	total: 2.83s	remaining: 4.73s
374:	learn: 0.5586053	total: 2.83s	remaining: 4.73s
375:	learn: 0.5585156	total: 2.84s	remaining: 4.72s
376:	learn: 0.5579184	total: 2.85s	remaining: 4.71s
377:	learn: 0.5578352	total: 2.86s	remaining: 4.7s
378:	learn: 0.5577227	total: 2.87s	remaining: 4.7s
379:	learn: 0.5576851	total: 2.87s	remaining: 4.68s
380:	learn: 0.5573191	total: 2.88s	remaining: 4.68s
381:	learn: 0.5571780	total: 2.88s	remaining: 4.67s
382:	learn: 0.5571391	total: 2.9s	remaining: 4.67s
383:	learn: 0.5569123	total: 2.92s	remaining: 4.69s
384:	learn: 0.5567230	total: 2.94s	remaining: 4.7s
385:	learn: 0.556

538:	learn: 0.5293125	total: 4.14s	remaining: 3.54s
539:	learn: 0.5292350	total: 4.15s	remaining: 3.53s
540:	learn: 0.5289046	total: 4.15s	remaining: 3.52s
541:	learn: 0.5288467	total: 4.16s	remaining: 3.52s
542:	learn: 0.5287310	total: 4.17s	remaining: 3.51s
543:	learn: 0.5284315	total: 4.17s	remaining: 3.5s
544:	learn: 0.5282696	total: 4.18s	remaining: 3.49s
545:	learn: 0.5282050	total: 4.19s	remaining: 3.48s
546:	learn: 0.5279718	total: 4.2s	remaining: 3.48s
547:	learn: 0.5279041	total: 4.2s	remaining: 3.47s
548:	learn: 0.5278244	total: 4.21s	remaining: 3.46s
549:	learn: 0.5277300	total: 4.22s	remaining: 3.45s
550:	learn: 0.5275380	total: 4.23s	remaining: 3.44s
551:	learn: 0.5273275	total: 4.23s	remaining: 3.44s
552:	learn: 0.5273274	total: 4.24s	remaining: 3.43s
553:	learn: 0.5272607	total: 4.25s	remaining: 3.42s
554:	learn: 0.5269970	total: 4.25s	remaining: 3.41s
555:	learn: 0.5269255	total: 4.26s	remaining: 3.4s
556:	learn: 0.5266407	total: 4.27s	remaining: 3.4s
557:	learn: 0.526

698:	learn: 0.5083984	total: 5.29s	remaining: 2.28s
699:	learn: 0.5083041	total: 5.3s	remaining: 2.27s
700:	learn: 0.5079807	total: 5.31s	remaining: 2.26s
701:	learn: 0.5077778	total: 5.32s	remaining: 2.26s
702:	learn: 0.5076374	total: 5.32s	remaining: 2.25s
703:	learn: 0.5075855	total: 5.33s	remaining: 2.24s
704:	learn: 0.5075455	total: 5.34s	remaining: 2.23s
705:	learn: 0.5074256	total: 5.34s	remaining: 2.23s
706:	learn: 0.5073444	total: 5.35s	remaining: 2.22s
707:	learn: 0.5070190	total: 5.36s	remaining: 2.21s
708:	learn: 0.5069779	total: 5.37s	remaining: 2.2s
709:	learn: 0.5068851	total: 5.37s	remaining: 2.19s
710:	learn: 0.5067720	total: 5.38s	remaining: 2.19s
711:	learn: 0.5065179	total: 5.39s	remaining: 2.18s
712:	learn: 0.5062843	total: 5.4s	remaining: 2.17s
713:	learn: 0.5061915	total: 5.41s	remaining: 2.17s
714:	learn: 0.5060868	total: 5.42s	remaining: 2.16s
715:	learn: 0.5060053	total: 5.42s	remaining: 2.15s
716:	learn: 0.5058568	total: 5.44s	remaining: 2.15s
717:	learn: 0.5

883:	learn: 0.4838591	total: 6.84s	remaining: 897ms
884:	learn: 0.4837652	total: 6.85s	remaining: 890ms
885:	learn: 0.4837099	total: 6.85s	remaining: 882ms
886:	learn: 0.4836846	total: 6.86s	remaining: 874ms
887:	learn: 0.4835897	total: 6.87s	remaining: 867ms
888:	learn: 0.4835014	total: 6.88s	remaining: 859ms
889:	learn: 0.4833955	total: 6.88s	remaining: 851ms
890:	learn: 0.4833322	total: 6.89s	remaining: 843ms
891:	learn: 0.4832841	total: 6.9s	remaining: 836ms
892:	learn: 0.4832173	total: 6.91s	remaining: 828ms
893:	learn: 0.4831664	total: 6.92s	remaining: 820ms
894:	learn: 0.4831181	total: 6.92s	remaining: 812ms
895:	learn: 0.4830362	total: 6.93s	remaining: 804ms
896:	learn: 0.4829355	total: 6.94s	remaining: 797ms
897:	learn: 0.4829106	total: 6.95s	remaining: 789ms
898:	learn: 0.4828943	total: 6.95s	remaining: 781ms
899:	learn: 0.4827268	total: 6.96s	remaining: 773ms
900:	learn: 0.4826845	total: 6.97s	remaining: 766ms
901:	learn: 0.4826076	total: 6.98s	remaining: 758ms
902:	learn: 0

In [7]:
res = resultado.sort_values(by='Train Score', ascending=False)
res

,Model,Window,Neutral,Train Score,Test Score,Params
0,Random Forest,2,1,0.566416,0.274725,"{'max_depth': 16, 'n_estimators': 74}"
0,Random Forest,4,1,0.557632,0.472727,"{'max_depth': 7, 'n_estimators': 18}"
2,LightGBM,4,1,0.557417,0.427273,"{'colsample_bytree': 0.9088676775709073, 'gamma': 7.414001957543007, 'learning_rate': 0.16820968606354786, 'max_depth': 23, 'min_child_weight': 15.673507772465655, 'n_estimators': 22, 'reg_alpha': 9.85264241993676, 'subsample': 0.8913232995911916}"
2,LightGBM,5,1,0.555514,0.555556,"{'colsample_bytree': 0.8602674036957442, 'gamma': 3.0840563581607174, 'learning_rate': 0.010839049209721499, 'max_depth': 21, 'min_child_weight': 3.066729325809611, 'n_estimators': 50, 'reg_alpha': 3.072774458863231, 'subsample': 0.8756438007444007}"
1,XGBoost,5,0,0.551759,0.525000,"{'colsample_bytree': 0.5434088812859564, 'gamma': 2.58629233456382, 'learning_rate': 0.28559671512335, 'max_depth': 13, 'min_child_weight': 2.9439293465813634, 'n_estimators': 78, 'reg_alpha': 2.239140116803334, 'subsample': 0.927324524690535}"
2,LightGBM,5,0,0.550105,0.850000,"{'colsample_bytree': 0.9293774240451573, 'gamma': 0.1880358219492706, 'learning_rate': 0.18314621241337253, 'max_depth': 23, 'min_child_weight': 6.5775980456296885, 'n_estimators': 87, 'reg_alpha': 0.18665565912707288, 'subsample': 0.9619360407173027}"
0,Random Forest,5,0,0.545220,0.600000,"{'max_depth': 2, 'n_estimators': 73}"
1,XGBoost,4,1,0.542738,0.418182,"{'colsample_bytree': 0.9163857490856366, 'gamma': 3.0752627304053815, 'learning_rate': 0.19092514431872143, 'max_depth': 17, 'min_child_weight': 6.725384390073773, 'n_estimators': 72, 'reg_alpha': 5.563245069385181, 'subsample': 0.8335989283502176}"
2,LightGBM,3,0,0.541386,0.522727,"{'colsample_bytree': 0.9688965699713962, 'gamma': 5.308573888138706, 'learning_rate': 0.2192988087711616, 'max_depth': 22, 'min_child_weight': 8.269454677556734, 'n_estimators': 14, 'reg_alpha': 2.2856612926778195, 'subsample': 0.9864655582749143}"
1,XGBoost,5,1,0.539481,0.400000,"{'colsample_bytree': 0.8951260772092376, 'gamma': 0.32632592938970806, 'learning_rate': 0.10889586913300252, 'max_depth': 18, 'min_child_weight': 4.122960111381978, 'n_estimators': 61, 'reg_alpha': 6.145568338465288, 'subsample': 0.8995893404802122}"


In [8]:
res.to_csv('resultados2.csv')